In [ ]:
# !pip install nii2png
import os
from pathlib import Path
import os
import pandas as pd
from cimaq_utils import absoluteFilePaths, str_inc, seq_eve, seq_odd 

fmriprep_dir = '/data/simexp/cimaq_preproc/fmriprep/'
events_dir = '/data/simexp/CIMAQ_AS_BIDS/'
atlases_dir = '../../nilearn_atlases/'
task, space, mdlt = 'memory', 'MNI152NLin2009cAsym', 'T1w'

anat_suffix = f'*_space-{space}_desc-preproc_{mdlt}.nii.gz'
bold_suffix = f'_task-{task}_space-{space}_desc-preproc_bold.nii.gz'
mask_suffix = f'_task-{task}_space-{space}_desc-brain_mask.nii.gz'

sub_ids = list(next(os.walk(fmriprep_dir)))[1]
sub_ids
fmri_paths = '\n'.join(sorted(str_inc([bold_suffix],
                                      list(absoluteFilePaths(fmriprep_dir)))))
fmri_paths.splitlines()[0]
# Path('/data/simexp/fnadeau/memory_bold_files.txt').write_text(fmri_paths)

In [ ]:
# os.mkdir('/data/simexp/fnadeau/cimaq_2d/')
# 'nhtlnbrlk.gg.nii.xz'.rsplit('.')
# v=180

# import nibabel
# from nilearn.image import load_img
# rot904d = lambda arr, x, z: numpy.rot90(arr[:, :, x, z])
# image_array = load_img(path00).get_data()
# rot904d(image_array)
divmod(270,90)

In [ ]:
path00='/data/simexp/fnadeau/cimaq_2d/sub-3002498/ses-V10/func/sub-3002498_ses-V10_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
# path00 = Path(path00)
# # dir(path00)
# # path00.split(path00.suffixes[0])[0]
# justname = str(path00).rstrip(''.join(path00.suffixes))

In [ ]:
def get_fmriprep_mask(fmri_path:Union[str,os.PathLike],
                      mask_ext:str='nii.gz', **kwargs):
    bids_patt = lambda p: f'(?<={p})[a-zA-Z0-9]*'
    prefixes = ['sub-','ses-','task-','space-']
    mask_sfx = '_'.join([pf+re.search(bids_patt(pf),
                                      os.path.basename(fmri_path)).group()
                         for pf in prefixes]+[f'desc-brain_mask.{mask_ext}'])
    return glob.glob(os.path.join(os.path.dirname(fmri_path), mask_sfx))[0]

def get_fmriprep_anat(fmri_path:Union[str,os.PathLike],
                      mdlt:str='T1w', ext:str='nii.gz',
                      **kwargs):
    space = '_space-'+re.search(f'(?<=_space-)[a-zA-Z0-9]*',
                      os.path.basename(fmri_path)).group()
    anat_suffix = f'*{space}_desc-preproc_{mdlt}.{ext}'
    return str(next(list(Path(fmri_path).parents)[2].rglob(anat_suffix)))

def get_events(fmri_path:Union[str,os.PathLike],
               events_dir:Union[str,os.PathLike])->str:
    sub_id, ses_id = Path(fmri_path).parts[-4:-2]
    globbed = glob.glob(os.path.join(events_dir,
                                  *Path(fmri_path).parts[-4:-2],
                                  '*events.tsv'))
    return [False if globbed == [] else globbed[0]][0]

        
def fetch_fmriprep_session(fmri_path:Union[str,os.PathLike],
                           events_dir:Union[str,os.PathLike],
                           strategy:str='Minimal',
                           anat_mod:str='T1w',
                           lc_kws:dict=None,
                           clean_kws:dict=None,
                           **kwargs):
    import load_confounds
    from inspect import getmembers
    from sklearn.utils import Bunch
    from pathlib import Path
    sub_id, ses_id = Path(fmri_path).parts[-4:-2]
    mask_path = get_fmriprep_mask(fmri_path)
    anat_path = get_fmriprep_anat(fmri_path)
    loader = dict(getmembers(load_confounds))[f'{strategy}']
    loader = [loader(**lc_kws) if lc_kws is not None
              else loader()][0]    
    event_path = get_events(fmri_path,events_dir)
    if event_path is False:
        return False
    else:
        return Bunch(**dict(sub_id=sub_id, ses_id=ses_id,
                            full_mask_path=mask_path,
                            fmri_path=fmri_path,
                            anat_path=anat_path,
                            event_path=event_path,
                            loader=loader,
                            confounds_strategy=strategy))

def load_fmriprep_session(fmri_path:Union[str,os.PathLike],
                          events_dir:Union[str,os.PathLike],
                          clean_kws:dict=None,
                          masker_kws:dict=None,
                          **kwargs):
    from nilearn import image as nimage
    from nilearn.masking import apply_mask, unmask
    from nilearn.signal import clean
    from sklearn.utils import Bunch
    from glob import glob
    from nilearn import image as nimage
    session = fetch_fmriprep_session(fmri_path,events_dir)
    fmri_img, mask_img, anat_img = \
        tuple(map(nimage.load_img,[session.fmri_path, session.full_mask_path,
                                   session.anat_path]))
    t_r = fmri_img.header.get_zooms()[-1]
    conf = session.loader.load(session.fmri_path)
    clean_defs = dict(standardize=False,
                      standardize_confounds=False,
                      high_pass=None, low_pass=None,
                      t_r=fmri_img.header.get_zooms()[-1],
                      ensure_finite=True)
    masker_defs = dict(mask_img=mask_img,
                       allow_overlap=True, t_r=t_r,
                       standardize_confounds=False,
                       resampling_target='mask')
    if masker_kws is not None:
        masker_defs.update(masker_kws)
    if clean_kws is not None:
        clean_defs.update(clean_kws)
    cleaned_fmri = unmask(clean(apply_mask(fmri_img, mask_img,
                                           smoothing_fwhm=8,
                                           dtype='f'),
                                confounds=conf,
                                **clean_defs),
                          mask_img)
    cleaned_fmri = nimage.new_img_like(fmri_img, cleaned_fmri.get_fdata(),
                               copy_header=True)
    events = pd.read_csv(session.event_path, sep='\t').iloc[1:,:]
    loaded = Bunch(**dict(fmri_img=fmri_img, cleaned_fmri=cleaned_fmri,
                          full_mask_img=mask_img, anat_img=anat_img,
                          clean_params=clean_defs, masker_params=masker_defs,
                          confounds=conf, events=events, t_r=t_r))
    loaded.update(session)
    return loaded

In [ ]:
# import numpy as np
# def rot90flip(data, rot_angle:int=None, scl, vol):
    
rot90flip=lambda a, s, v: np.rot90(a[:,:,s,v])
rot180flip=lambda a, s, v: np.rot90(np.rot90(a[:,:,s,v]))
rot270flip=lambda a, s, v: np.rot90(np.rot90(np.rot90(a[:,:,s,v])))
rot_dict = {90: rot90flip, 180: rot180flip, 270: rot270flip}
rot_dict[90]([0,1,2], 2, 3)
# np.rot90([[0,1,2]][:, :, 2, 33])
# import scipy
# dir(scipy)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [16]:
# src = '../../flat_bold_png/sub-3002498_ses-V10_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold/sub-3002498_ses-V10_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold_t306_z46.png'


# imageio.imread(src).shape
help(np.repeat)

# Image.open(src)._Image__transformer()
# dir(img._Image__transformer)
# skimage.show(input_image)

Help on function repeat in module numpy:

repeat(a, repeats, axis=None)
    Repeat elements of an array.
    
    Parameters
    ----------
    a : array_like
        Input array.
    repeats : int or array of ints
        The number of repetitions for each element.  `repeats` is broadcasted
        to fit the shape of the given axis.
    axis : int, optional
        The axis along which to repeat values.  By default, use the
        flattened input array, and return a flat output array.
    
    Returns
    -------
    repeated_array : ndarray
        Output array which has the same shape as `a`, except along
        the given axis.
    
    See Also
    --------
    tile : Tile an array.
    
    Examples
    --------
    >>> np.repeat(3, 4)
    array([3, 3, 3, 3])
    >>> x = np.array([[1,2],[3,4]])
    >>> np.repeat(x, 2)
    array([1, 1, 2, 2, 3, 3, 4, 4])
    >>> np.repeat(x, 3, axis=1)
    array([[1, 1, 1, 2, 2, 2],
           [3, 3, 3, 4, 4, 4]])
    >>> np.repeat(x, [1, 2], ax

In [29]:
from pathlib import Path
cimaq_stims = list(map(str, list(Path('../../WMStim').rglob('*.bmp'))))

In [65]:
def preprocess_image(src:Union[str,os.PathLike]):
    # Open image
#     input_image = PIL.Image.fromarray(np.uint8(PIL.Image.open(src)))
#     input_image = PIL.Image.fromarray(PIL.Image.open(src))
    input_image = PIL.Image.open(src)
    # Preprocess image
    preprocess = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                          std=[0.229, 0.224, 0.225])])
    return preprocess(input_image).unsqueeze(0)
preprocess_image(cimaq_stims[0])

tensor([[[[2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          ...,
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
          [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489]],

         [[2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          ...,
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
          [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286]],

         [[2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
          [2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
          [2.6400, 2.6400, 2.6400,  ..., 2

In [62]:
help(imageio.imread)

Help on function imread in module imageio.core.functions:

imread(uri, format=None, **kwargs)
    imread(uri, format=None, **kwargs)
    
    Reads an image from the specified file. Returns a numpy array, which
    comes with a dict of meta data at its 'meta' attribute.
    
    Note that the image data is returned as-is, and may not always have
    a dtype of uint8 (and thus may differ from what e.g. PIL returns).
    
    Parameters
    ----------
    uri : {str, pathlib.Path, bytes, file}
        The resource to load the image from, e.g. a filename, pathlib.Path,
        http address or file object, see the docs for more info.
    format : str
        The format to use to read the file. By default imageio selects
        the appropriate for you based on the filename and its contents.
    kwargs : ...
        Further keyword arguments are passed to the reader. See :func:`.help`
        to see what arguments are available for a particular format.



In [79]:
# len(list(filter(lambda x: x.endswith('.bmp'), cimaq_stims)))==len(cimaq_stims)
sorted(cimaq_stims)[6:]

['../../WMStim/Kitchen_apron.bmp',
 '../../WMStim/animal_camel_new.bmp',
 '../../WMStim/animal_cheetah.bmp',
 '../../WMStim/animal_chimpanzee.bmp',
 '../../WMStim/animal_dalmation_old.bmp',
 '../../WMStim/animal_dolphin_old.bmp',
 '../../WMStim/animal_farm pig.bmp',
 '../../WMStim/animal_frog.bmp',
 '../../WMStim/animal_giraffe.bmp',
 '../../WMStim/animal_gorilla.bmp',
 '../../WMStim/animal_hen.bmp',
 '../../WMStim/animal_hippo_new.bmp',
 '../../WMStim/animal_horse.bmp',
 '../../WMStim/animal_lion_new.bmp',
 '../../WMStim/animal_monarch_old.bmp',
 '../../WMStim/animal_mouse.bmp',
 '../../WMStim/animal_ostrich.bmp',
 '../../WMStim/animal_parrot.bmp',
 '../../WMStim/animal_penguin.bmp',
 '../../WMStim/animal_pigeon.bmp',
 '../../WMStim/animal_polar bear_old.bmp',
 '../../WMStim/animal_seal.bmp',
 '../../WMStim/animal_sheep.bmp',
 '../../WMStim/animal_siamese cat.bmp',
 '../../WMStim/animal_tortoise.bmp',
 '../../WMStim/animal_white tiger.bmp',
 '../../WMStim/animal_zebra_old.bmp',
 '../.

In [80]:
model = GoogLeNet.from_pretrained("googlenet")

def get_features(src, model):
#     input_tensor = preprocess_image(src)
#     model.eval()
    return model.extract_features(preprocess_image(src))
features = [get_features(src, model) for src in sorted(cimaq_stims)[6:]]
labels = list(map(os.path.basename, sorted(cimaq_stims)[6:]))
# list(get_features(cimaq_stims[x], model) for x in range(len(cimaq_stims)))
# help(model.eval)
# eval mode shape == torch.Size([1, 1024, 7, 7])


Loaded pretrained weights for googlenet


In [48]:
apath = '../../flat_bold_png/sub-3002498_ses-V10_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold/sub-3002498_ses-V10_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold_t306_z46.png'
from googlenet_pytorch import GoogLeNet
from typing import Union
# model = GoogLeNet.from_pretrained("ImageNet")

import numpy as np
import PIL
import skimage
import imageio
import json
import os
import torch
from io import BytesIO
import torchvision.transforms as transforms
from PIL import Image
from keras.applications.vgg16 import VGG16
from googlenet_pytorch import GoogLeNet 


def load_inputs(src:Union[str,os.PathLike]):
    # Open image
#     input_image = PIL.Image.open(BytesIO(imageio.imread(src)))
#     model = VGG16()
    model = GoogLeNet.from_pretrained("googlenet")
    input_image = PIL.Image.fromarray(np.stack(np.repeat(np.uint8(PIL.Image.open(src)),3, axis=0)))
    # Preprocess image
    preprocess = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                          std=[0.229, 0.224, 0.225])])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)  # create a mini-batch as expected by the model

    # Load class names
#     labels_map = json.load(open("labels_map.txt"))
#     labels_map = [labels_map[str(i)] for i in range(1000)]

    # Feature extraction
    # ... image preprocessing as in the classification example ...
    # inputs = torch.randn(1, 3, 224, 224)
#     print(inputs.shape) # torch.Size([1, 3, 224, 224])

    features = model.extract_features(input_batch)
    print(features.shape) # torch.Size([1, 1024, 7, 7])

    # Classify with GoogLeNet
    model.eval()

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to("cuda")
        model.to("cuda")

    with torch.no_grad():
        logits = model(input_batch)
        preds = torch.topk(logits, k=5).indices.squeeze(0).tolist()
        print("-----")
        print(f'Predictions: {preds}')
        for idx in preds:
#             label = labels_map[idx]
            prob = torch.softmax(logits, dim=1)[0, idx].item()
#             print(f"{label:<75} ({prob * 100:.2f}%)")
    return input_tensor, input_batch, features, prob
test_tensors, test_batch, test_features, test_prob = load_inputs(cimaq_stims[0])

Loaded pretrained weights for googlenet
torch.Size([1, 1024, 7, 7])


TypeError: eval() takes 1 positional argument but 2 were given

In [44]:
test_prob

0.006411508657038212

In [36]:
import torch
from googlenet_pytorch import GoogLeNet
from numpy import asarray
model = GoogLeNet.from_pretrained('ImageNet')
apath = '../../flat_bold_png/sub-3002498_ses-V10_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold/sub-3002498_ses-V10_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold_t306_z46.png'
from typing import Union
import json
import torchvision.transforms as transforms
from PIL import Image

from googlenet_pytorch import GoogLeNet 
# ... image preprocessing as in the classification example ...
# inputs = torch.randn(1, 3, 224, 224)
# inputs = sorted(map(str,list(Path(testsess).iterdir())))
# input_image = asarray(Image.open(apath))
preprocess = transforms.Compose([transforms.Resize(256),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                      std=[0.229, 0.224, 0.225])])
input_tensor = preprocess(Image.open(apath))
print(input_tensor.shape) # torch.Size([1, 3, 224, 224])

features = model.extract_features(input_tensor)
torch.Size([1, 1024, 7, 7])
print(features.shape)

ValueError: model_name should be one of: googlenet

In [30]:
dir(Image.open(apath))

['_Image__transformer',
 '_PngImageFile__prepare_idat',
 '__array_interface__',
 '__class__',
 '__copy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_close_exclusive_fp_after_loading',
 '_copy',
 '_crop',
 '_dump',
 '_ensure_mutable',
 '_exclusive_fp',
 '_exif',
 '_expand',
 '_get_safe_box',
 '_getexif',
 '_min_frame',
 '_new',
 '_open',
 '_repr_png_',
 '_seek_check',
 '_size',
 '_text',
 'alpha_composite',
 'category',
 'close',
 'convert',
 'copy',
 'crop',
 'custom_mimetype',
 'decoderconfig',
 'decodermaxblock',
 'draft',
 'effect_spread',
 'entropy',
 'filename',
 'filter',
 'format',
 'format_description',
 'f

In [ ]:
# !pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
from googlenet_pytorch import GoogLeNet
model = GoogLeNet.from_pretrained("googlenet")

import json

import torch
import torchvision.transforms as transforms
from PIL import Image

from googlenet_pytorch import GoogLeNet 

In [ ]:
!pip install -U IProgress

In [4]:
# 
import os
os.getcwd()

'/data/simexp/fnadeau/cimaq_memory/extract_features'

In [7]:
len(os.listdir('sub-3163875_ses-V03_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold'))

18600

In [18]:
flatdir = '../../flat_bold_png/'
from itertools import starmap
from pathlib import Path
testsess = sorted(Path(flatdir).iterdir())[0]

In [24]:
str(list(Path(testsess).iterdir())[0])

'../../flat_bold_png/sub-3002498_ses-V10_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold/sub-3002498_ses-V10_task-memory_space-MNI152NLin2009cAsym_desc-preproc_bold_t306_z46.png'